In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
import glob
import pandas as pd

import sys
sys.path.append("/content/drive/My Drive/Colab Notebooks/")  

import numpy as np
from train_brain import pre_cut, scale, one_hot_encode
import train_brain
from augmentations import full_augment
# rated individual: [gene_a, gene_b,..., classification loss]
# rated population: [rated individual 0, rated individual 1, ..]


def mock_rate_loss(individual) -> float:
    # fitness = np.sum(individual)
    gene_count = individual.shape[0]
    unfitness = 0
    for i in range(gene_count):
        unfitness += (-1)*(abs(individual[gene_count-1]-0.5))
    return unfitness


def rate_loss(individual) -> float:
    loss = train_brain.train_test_individual(individual, x_train, y_train, x_test, y_test)
    return loss


def add_loss(unrated_individual: np.ndarray) -> np.ndarray:
    loss = rate_loss(unrated_individual)
    rated_individual = np.concatenate((unrated_individual, loss), axis=None)
    return rated_individual


def create_rated_individual(gene_count) -> np.ndarray:
    individual = np.random.rand(gene_count)
    rated_individual = add_loss(individual)
    return rated_individual


# rated population is already sorted , but sorting again is still required when adding new individuals
def create_rated_population(population_count: int, gene_count: int) -> np.ndarray:
    population = np.empty((0, gene_count + 1))
    for i in range(population_count):
        individual = create_rated_individual(gene_count)
        population = np.vstack([population, individual])

    sorted_population = population[population[:, gene_count].argsort()]
    return sorted_population


# sorts population according to fitness in desc order and returns the $count best individuals
def get_best_individuals(rated_population: np.ndarray, count: int) -> np.ndarray:
    gene_count = rated_population.shape[1] - 1
    sorted_population = rated_population[rated_population[:, gene_count].argsort()]

    best_individuals = np.empty((0, gene_count+1))

    for i in range(count):
        ith_best_individual = sorted_population[i]
        best_individuals = np.vstack([best_individuals, ith_best_individual])
    return best_individuals


# combines the same trait from multiple parents
def combine_genes_avg(genes: np.ndarray) -> int:
    gene_result = 0
    genes = genes.flatten()
    parent_count = len(genes)  # each gene is from a different parent
    for gene in genes:
        gene_result += gene / parent_count
    #gene_result = round(gene_result)
    return gene_result


def mutate(gene: float, chance: float) -> float:
    if np.random.rand() < chance:
        gene = gene * np.random.rand()
    return gene


# combines the same trait from multiple parents
def combine_genes_rand_weight(genes: np.ndarray) -> int:
    gene_result = 0
    genes = genes.flatten()
    parent_count = len(genes)

    weights = np.random.rand(parent_count)
    normalized_weights = weights / np.sum(weights)

    for index, gene in enumerate(genes):
        gene_result += gene * normalized_weights[index]

    gene_result = mutate(gene_result, 0.1)
    return gene_result


def make_rated_children(parents: np.ndarray, count: int) -> np.ndarray:
    gene_count = parents.shape[1] - 1  # subtract fitness score
    child = np.zeros(gene_count)
    children = np.empty((0, parents.shape[1]))

    for ch in range(count):
        for gene in range(gene_count):
            child[gene] = combine_genes_rand_weight(parents[:, [gene]])

        rated_child = add_loss(child)
        children = np.vstack([children, rated_child])

    return children

  
def load_data(data_dir, pre_cut_start, pre_cut_length):
    labels = np.empty((0, 3))
    recordings = np.empty((0, pre_cut_length, 8))

    all_train_files = glob.glob(data_dir + "/*.csv")
    print(all_train_files)
    for index, freq_suffix in enumerate(freq_suffices):
        current_freq_files = [s for s in all_train_files if freq_suffix in s]
        for current_freq_file in current_freq_files:
            recording = pd.read_csv(current_freq_file, header=None)
            recording = pre_cut(recording, pre_cut_start, pre_cut_length)
            recording = scale(recording)
            recordings = np.append(recordings, [recording], axis=0)

            label = one_hot_encode(index, len(freq_suffices))
            labels = np.append(labels, [label], axis=0)
    return recordings, labels


  
def init_data(train_data_dir, test_data_dir):
    my_x_train, my_y_train = load_data(train_data_dir, pre_cut_start=0, pre_cut_length=6000)
    x_train_aug, y_train_aug = full_augment(my_x_train, my_y_train, 2)

    my_x_test, my_y_test = load_data(test_data_dir, pre_cut_start=1000, pre_cut_length=5000)

    return x_train_aug, y_train_aug, my_x_test, my_y_test

  

epochs = 30
population_size = 1000
my_gene_count = 5
my_parent_count = 5
my_children_count = 20

my_train_data_dir = "/content/drive/My Drive/Colab Notebooks/data/training_data"
my_test_data_dir = "/content/drive/My Drive/Colab Notebooks/data/test_data"
freq_suffices = ["_8Hz", "_14Hz", "_28Hz"]



x_train, y_train, x_test, y_test = init_data(my_train_data_dir,my_test_data_dir)

my_initial_population = create_rated_population(population_size, my_gene_count)
besties = get_best_individuals(my_initial_population, my_parent_count)

for e in range(epochs):
    print("==== EPOCH " + str(e) + "==============")
    my_children = make_rated_children(besties, my_children_count)
    besties = get_best_individuals(my_children, my_parent_count)
    print(besties[0][-1])

Using TensorFlow backend.


['/content/drive/My Drive/Colab Notebooks/data/training_data/Subj4_14Hz_Trial2.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj1_8Hz_Trial3.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj3_14Hz_Trial5.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj1_8Hz_Trial5.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj3_28Hz_Trial3.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj2_8Hz_Trial5.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj2_8Hz_Trial2.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj3_14Hz_Trial2.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj3_14Hz_Trial3.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj3_28Hz_Trial2.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj3_8Hz_Trial2.csv', '/content/drive/My Drive/Colab Notebooks/data/training_data/Subj2_28Hz_Trial5.cs